<h3> Imports </h3>

In [ ]:
import numpy as np
import pandas as pd
import ebooklib
import re
import nltk
from nltk.util import ngrams
from ebooklib import epub
from bs4 import BeautifulSoup
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.util import ngrams
import gensim 
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

<h3>Reading the epub file : </h3>

In [ ]:
def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

In [ ]:
blacklist = [   '[document]',   'noscript', 'header',   'html', 'meta', 'head','input', 'script',   ]

In [ ]:
def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output

In [ ]:
def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output

In [ ]:
def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

In [ ]:
#epub file as a list
out=epub2text('The-Art-of-Digital-Marketing-The-Definitive-Guide-to-Creating-Strategic-Targeted-and-Measurable-Online-Campaigns.epub')

<h3> Data Munging </h3>

In [ ]:
def to_string(text) :
    return ' '.join(text)
text = to_string(out)

In [ ]:
#saving the text in a newfile
with open("text.cvs", "w") as text_file:
    print(f"text: {text}", file=text_file)

In [ ]:
#reading each line
with open("text.cvs", "r") as text:
    text = text.readlines()

In [ ]:
def cleaning(text) :
    #removing newlines(\n)
    text = [l.strip('\n') for l in lines]
    text = [l.lower() for l in text]
    text = [token.lemma_ for token in doc if not token.is_stop]

    if len(text) > 2:
        return " ".join(text)

In [ ]:
text = to_string(text)
re.sub("[^A-Za-z']+",' ',text)

In [ ]:
import spacy
#using spacy
nlp = spacy.load("en_core_web_sm")
text = ''.join(text)
doc = nlp(text)

In [ ]:
from time import time 

t = time()

text = [cleaning(doc) for doc in nlp.pipe(text, batch_size=50, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
from gensim.models.phrases import Phrases, Phraser
phrases = Phrases(text, min_count=30, progress_per=10000)

In [ ]:
#ngrams
sentences = bigram[text]

In [ ]:
#stemming
from nltk.stem.snowball import SnowballStemmer


In [ ]:
#tokenizing the text per sentence 
text = re.compile('[.!?] ').split(text)

In [ ]:
#removing empty spaces 
text = re.sub(r"\s+"," ", text, flags = re.I)

In [ ]:
#deleting stop words
text = [token.text for token in doc if not token.is_stop]

In [ ]:
#3grams
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]
  
extract_ngrams(text,3)

In [ ]:
#from spacy.lang.en import English

#with open("skills.cvs") as skills:
    #skills.read()

#nlp = English()
#doc = nlp(doc)

#from spacy.matcher import PhraseMatcher

#matcher = PhraseMatcher(nlp.vocab)

#[nlp(skills) for skill in skills]
#patterns = list(nlp.pipe(skills))
#matcher.add("skills", None, *patterns)

#matches = matcher(doc)
#print([doc[start:end] for match_id, start, end in matches])

<h3>Training</h3>

In [ ]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 

In [ ]:
model = gensim.models.Word2Vec(
    text,
    size=150,
    window=5,
    min_count=2,
    iter=10)